In [3]:
import os
import json
import pandas as pd
from numpy import array
from datasets import Dataset, DatasetDict, load_from_disk
from csv_to_arrow import csv_to_arrow, modify_filename

In [2]:
parent_dir = os.path.dirname(os.getcwd())
train_filename = os.path.join(parent_dir, "data", "csv", "train_dataset.csv")
valid_filename = os.path.join(parent_dir, "data", "csv", "train_validation_dataset.csv")

train_df = pd.read_csv(train_filename)
valid_df = pd.read_csv(valid_filename)

# type str -> object(dict)로 변환
train_df["answers"] = train_df["answers"].apply(lambda x: eval(x))
valid_df["answers"] = valid_df["answers"].apply(lambda x: eval(x))

len(train_df), len(valid_df)

(3952, 240)

In [13]:
# train/val 통합
concated_df = pd.concat([train_df, valid_df],ignore_index=True)
concated_dataset = Dataset.from_pandas(concated_df)
concated_dataset.save_to_disk("concated")

Saving the dataset (1/1 shards): 100%|██████████| 4192/4192 [00:00<00:00, 461628.92 examples/s]


In [21]:
# train/val 재분할
sampled_df = train_df.sample(n=240, random_state=2024)
remaining_df = train_df.drop(sampled_df.index)

new_train_df = pd.concat([remaining_df, valid_df])
new_valid_df = sampled_df

output_dir = "split_seed_2022"
new_train_filename =  os.path.join(output_dir, "train_dataset.csv")
new_valid_filename = os.path.join(output_dir,"train_validation_dataset.csv")

new_train_df.to_csv(new_train_filename, index=False)
new_valid_df.to_csv(new_valid_filename, index=False)

len(new_train_df), len(new_valid_df)

(3952, 240)

In [22]:
from csv_to_arrow import csv_to_arrow,modify_filename

csv_to_arrow(new_train_filename, new_valid_filename, output_dir)
modify_filename(output_dir)

Saving the dataset (1/1 shards): 100%|██████████| 240/240 [00:00<00:00, 81469.16 examples/s]

Augmented dataset saved to split_seed_2022
